In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
model = PhysNet()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4), loss=np_loss)
model(np.random.random((4, 128, 32, 32, 3)));
model.summary()

Model: "phys_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (4, 128, 32, 32, 16)   │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (4, 128, 16, 16, 32)   │        13,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (4, 128, 16, 16, 64)   │        55,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (4, 64, 8, 8, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (4, 64, 8, 8, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_5 (Sequential)       │ (4, 32, 4, 4, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (4, 32, 4, 4, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ (4, 32, 2, 2, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_8 (Sequential)       │ (4, 32, 2, 2, 64)      │       110,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_9 (Sequential)       │ (4, 64, 2, 2, 64)      │        16,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_10 (Sequential)      │ (4, 128, 2, 2, 64)     │        16,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_9 (Conv3D)               │ (4, 128, 1, 1, 1)      │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d (MaxPooling3D)    │ (4, 128, 16, 16, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_1 (MaxPooling3D)  │ (4, 64, 8, 8, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling3d               │ (4, 128, 1, 1, 64)     │             0 │
│ (AveragePooling3D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (4, 128)               │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769,825 (2.94 MB)

 Trainable params: 768,577 (2.93 MB)

 Non-trainable params: 1,248 (4.88 KB)

In [3]:
batch_size = 128

rlap_tape = "/root/ssd_cache/rppg_training_data/rlap_128x32x32_all"

train_rlap = load_datatape(rlap_tape, fold='train', batch=batch_size)
valid_rlap = load_datatape(rlap_tape, fold='val', extended_hr='False', batch=batch_size)

#train_rlap = DatatapeMonitor(train_rlap)
training_set, validation_set = KerasDataset(train_rlap), KerasDataset(valid_rlap)

training_set = training_set.apply_fn(compress_aug)

rlap_128x32x32_all             datatape has been loaded.    33024 items total. fold=train selected
rlap_128x32x32_all             datatape has been loaded.     4096 items total. fold=val&extended_hr=False selected


In [4]:
#model.fit(training_set, validation_data=validation_set, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/physnet.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(model, training_set, validation_set, epochs=20, check_point_path='../weights/physnet.weights.h5')
model.load_weights('../weights/physnet.weights.h5')

Training:   0%|          | 0/258 [00:00<?, ?it/s]

Validating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch   1:  Training loss:0.9953	Validation loss:0.9949	


Training:   0%|          | 0/258 [00:00<?, ?it/s]

Validating:   0%|          | 0/32 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.1654	Validation loss:0.1961	


Training:   0%|          | 0/258 [00:00<?, ?it/s]

Validating:   0%|          | 0/32 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.1637	Validation loss: 0.196	


In [5]:
model = PhysNet()
model(np.random.random((4, 128, 32, 32, 3)));
model.load_weights('../weights/physnet.weights.h5')

In [6]:
eval_on_dataset(dataset_H5_mmpd, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNet_RLAP_MMPD.h5')
get_metrics('../results/PhysNet_RLAP_MMPD.h5', dropped='False')

  0%|          | 0/660 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,14.352±0.197,22.039±3.295,16.55±0.223,0.1849
Whole,11.38±0.474,16.646±4.5,12.905±0.505,0.32723


In [7]:
eval_on_dataset(dataset_H5_cohface, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNet_RLAP_COHFACE.h5', fps=30)
get_metrics('../results/PhysNet_RLAP_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,12.897±0.325,18.953±4.042,18.553±0.477,0.09485
Whole,9.969±0.742,13.77±5.355,14.015±1.053,0.16259


In [8]:
eval_on_dataset(dataset_H5_pure, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNet_RLAP_PURE.h5')
get_metrics('../results/PhysNet_RLAP_PURE.h5')

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,7.621±0.582,17.794±5.924,10.397±0.857,0.66256
Whole,6.489±1.468,13.013±7.589,8.618±2.074,0.83336


In [9]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(model), 128, (32, 32), step=3, batch=16, save='../results/PhysNet_RLAP_UBFC.h5')
get_metrics('../results/PhysNet_RLAP_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.266±0.092,2.458±1.125,1.3±0.099,0.98964
Whole,0.592±0.106,0.908±0.555,0.587±0.106,0.9988
